# Scenario 1 - Create a New User in Azure using Power-Automate/Forms/Jupyter Notebooks


## Create User
HR User fills out new employee creation form

### Power Automate Form to grab the form response and insert response into Excel Table

### Query the Excel file stored on One-Drive via Jupyter Notebooks and REST API

#### Build the JSON payload and headers

In [1]:
$host.UI.RawUI.BufferSize = [System.Management.Automation.Host.Size]::new(200, 50) ## Corrects the output
$userDetails = @{ Workflow = "list" }

$userDetails = ($userDetails | ConvertTO-JSON); 

$headers = @{
    'Content-Type' = "application/json"
}

Write-Host -ForegroundColor Yellow "JSON Payload for REST Call"; Write-Host $userDetails

The below script needs to be able to find the current output cell; this is an easy method to get it.

JSON Payload for REST Call
{
  "Workflow": "list"
}


#### Query the custom Power-Automate Flow API for a list of users

In [2]:
$Response = (Invoke-RestMethod `
    -method POST `
    -headers $headers `
    -body $userDetails `
    -uri "https://prod-19.westcentralus.logic.azure.com:443/workflows/10456e974ab54633992cb73b36449ad8/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=EzbdUgRX3GoGjOLlzlmZxTmNjHmEleAuWJOlohC9ILY").value
    
$Response | FT


@odata.etag ItemInternalId                       First Name Last Name Organization Job Title E-mail address            Form Filled out HR Paperwork Computer assigned
----------- --------------                       ---------- --------- ------------ --------- --------------            --------------- ------------ -----------------
            70b48e14-883b-4d05-8582-a3b16a825114 Sarah      David     Finance      Intern    sara.david@jingtoso.com   Completed       Completed    Completed
            410fcc3c-6a16-4ee5-840d-669665082cde Bob        Smith     IT           Peon      bob@jingtoso.com          Completed       Completed    Not Started
            d564504e-e43f-497f-a8cc-f2dbc3daf44b Steve      Smith     Whatever     Marketing steve.smith@microsoft.com Completed       Completed    Completed



### Get New-Users Tracker Status 

In [3]:
$filteredColumns = @('E-mail address','Form Filled out','HR Paperwork',"Computer Assigned",'MFA Number setup', 'O365 License assigned', 'AD Group assignment')
$Response | Select-Object -Property $filteredColumns | FT


E-mail address            Form Filled out HR Paperwork Computer assigned MFA Number setup O365 License assigned AD Group assignment
--------------            --------------- ------------ ----------------- ---------------- --------------------- -------------------
sara.david@jingtoso.com   Completed       Completed    Completed         Completed        Completed             Completed
bob@jingtoso.com          Completed       Completed    Not Started       Not Started      Not Started           Not Started
steve.smith@microsoft.com Completed       Completed    Completed         Completed        Not Started           Not Started



#### Clean up table to HTML

In [4]:
$htmlCodeTable = @"
<TABLE BORDER=10>
<TR>
    <th style="text-align:Center">User</th>
    <th style="text-align:Center">Form Filled Out</th>
    <th style="text-align:Center">HR Paperwork Completed</th>
    <th style="text-align:Center">Computer Assigned</th>
    <th style="text-align:Center">MFA Number Setup</th>
    <th style="text-align:Center">License Assigned</th>
    <th style="text-align:Center">AD Group Assignment</th>
"@

function status {
    param ($status)
    if ($status -eq 'Completed') {
        return "<font color=green><b>"
    } else {
        return "<font color=red><b>"
    }
}

ForEach ($item in $Response) {
    $insertCode = "<TR>
        <TD> $($item.'First Name') $($item.'Last Name') ($($item.'E-mail address'))
        <TD> $(status -status $item.'Form Filled Out') $($item.'Form Filled Out')
        <TD> $(status -status $item.'HR Paperwork') $($item.'HR Paperwork')
        <TD> $(status -status $item.'Computer Assigned') $($item.'Computer Assigned')
        <TD> $(status -status $item.'MFA Number Setup') $($item.'MFA Number Setup')
        <TD> $(status -status $item.'O365 License assigned') $($item.'O365 License assigned')
        <TD> $(status -status $item.'AD Group assignment') $($item.'AD Group assignment')
    "
    $htmlCodeTable = $htmlCodeTable + $insertCode
}

$htmlCodeTable | Out-Display

User 
 Form Filled Out 
 HR Paperwork Completed 
 Computer Assigned 
 MFA Number Setup 
 License Assigned 
 AD Group Assignment 
 Sarah David (sara.david@jingtoso.com)
 Completed
 Completed
 Completed
 Completed
 Completed
 Completed
 
 Bob Smith (bob@jingtoso.com)
 Completed
 Completed
 Not Started
 Not Started
 Not Started
 Not Started
 
 Steve Smith (steve.smith@microsoft.com)
 Completed
 Completed
 Completed
 Completed
 Not Started
 Not Started

### Filter on specific User 

In [5]:
$Response | Where-Object {$_.'First Name' -eq 'Bob'} | ConvertTo-JSON

{
  "@odata.etag": "",
  "ItemInternalId": "410fcc3c-6a16-4ee5-840d-669665082cde",
  "First Name": "Bob",
  "Last Name": "Smith",
  "Organization": "IT",
  "Job Title": "Peon",
  "E-mail address": "bob@jingtoso.com",
  "Form Filled out": "Completed",
  "HR Paperwork": "Completed",
  "Computer assigned": "Not Started",
  "MFA Number setup": "Not Started",
  "O365 License assigned": "Not Started",
  "AD Group assignment": "Not Started"
}


In [6]:
$userDetails = @{
  "Workflow" = "update"
  "E-mail address" = "steve.smith@microsoft.com"
  "MFA Number Setup" =  "Completed"
}
$userDetails = ($userDetails | ConvertTO-JSON); 

$headers = @{
    'Content-Type' = "application/json"
}
Write-Host -ForegroundColor Yellow "JSON Payload for REST Call"; Write-Host $userDetails

JSON Payload for REST Call
{
  "E-mail address": "steve.smith@microsoft.com",
  "MFA Number Setup": "Completed",
  "Workflow": "update"
}


#### Query the custom Power-Automate Flow API for a list of users

In [7]:
$Response = (Invoke-RestMethod `
    -method POST `
    -headers $headers `
    -body $userDetails `
    -uri "https://prod-19.westcentralus.logic.azure.com:443/workflows/10456e974ab54633992cb73b36449ad8/triggers/manual/paths/invoke?api-version=2016-06-01&sp=%2Ftriggers%2Fmanual%2Frun&sv=1.0&sig=EzbdUgRX3GoGjOLlzlmZxTmNjHmEleAuWJOlohC9ILY").value
    
$Response | FT


@odata.etag ItemInternalId                       First Name Last Name Organization Job Title E-mail address            Form Filled out HR Paperwork Computer assigned
----------- --------------                       ---------- --------- ------------ --------- --------------            --------------- ------------ -----------------
            61f0a656-2648-4624-8437-834d1684ab90 Sarah      David     Finance      Intern    sara.david@jingtoso.com   Completed       Completed    Completed
            aa26a36b-5a41-4951-b685-e585dc4d7887 Bob        Smith     IT           Peon      bob@jingtoso.com          Completed       Completed    Not Started
            1c18f8ad-6ec3-4582-a413-805563f08af1 Steve      Smith     Whatever     Marketing steve.smith@microsoft.com Completed       Completed    Completed



In [8]:
$Response


@odata.etag           : 
ItemInternalId        : 61f0a656-2648-4624-8437-834d1684ab90
First Name            : Sarah
Last Name             : David
Organization          : Finance
Job Title             : Intern
E-mail address        : sara.david@jingtoso.com
Form Filled out       : Completed
HR Paperwork          : Completed
Computer assigned     : Completed
MFA Number setup      : Completed
O365 License assigned : Completed
AD Group assignment   : Completed

@odata.etag           : 
ItemInternalId        : aa26a36b-5a41-4951-b685-e585dc4d7887
First Name            : Bob
Last Name             : Smith
Organization          : IT
Job Title             : Peon
E-mail address        : bob@jingtoso.com
Form Filled out       : Completed
HR Paperwork          : Completed
Computer assigned     : Not Started
MFA Number setup      : Not Started
O365 License assigned : Not Started
AD Group assignment   : Not Started

@odata.etag           : 
ItemInternalId        : 1c18f8ad-6ec3-4582-a413-805563f08a